In [1]:
!nvidia-smi

Tue Apr 16 00:25:46 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA H100 80GB HBM3          On  | 00000000:18:00.0 Off |                    0 |
| N/A   32C    P0              72W / 700W |      4MiB / 81559MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
# fix numpy in colab
import numpy
from IPython.display import clear_output


#!huggingface-cli download lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo --quiet --cache-dir $TMP_DIR --local-dir Mixtral-8x7B-Instruct-v0.1-offloading-demo1


In [4]:

import os, sys
script_dir = os.getcwd()
module_path = script_dir
for _ in range(1):
    module_path = os.path.abspath(os.path.join(module_path, '../'))
    if module_path not in sys.path:
        sys.path.insert(0,module_path)
        
sys.path.append("mixtral-offloading")
import torch
from torch.nn import functional as F
from hqq.core.quantize import BaseQuantizeConfig
from huggingface_hub import snapshot_download
from IPython.display import clear_output
from tqdm.auto import trange
from transformers import AutoConfig, AutoTokenizer
from transformers.utils import logging as hf_logging
import time
import gc
from src.build_model import OffloadConfig, QuantConfig, build_model

In [5]:
# This will reload the imported modules (e.g. get_decode_model_characterstics) every time you execute the jupyter cells, so that you don't need to restart the notebook after updating the source codes.
%load_ext autoreload
%autoreload 2  

In [6]:
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"
quantized_model_name = "lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo"
state_path = "Mixtral-8x7B-Instruct-v0.1-offloading-demo1"

config = AutoConfig.from_pretrained(quantized_model_name)

device = torch.device("cuda:0")

##### Change this to 5 if you have only 12 GB of GPU VRAM #####
offload_per_layer = 4
# offload_per_layer = 5
###############################################################

num_experts = config.num_local_experts

offload_config = OffloadConfig(
    main_size=config.num_hidden_layers * (num_experts - offload_per_layer),
    offload_size=config.num_hidden_layers * offload_per_layer,
    buffer_size=4,
    offload_per_layer=offload_per_layer,
)


attn_config = BaseQuantizeConfig(
    nbits=4,
    group_size=64,
    quant_zero=True,
    quant_scale=True,
)
attn_config["scale_quant_params"]["group_size"] = 256


ffn_config = BaseQuantizeConfig(
    nbits=2,
    group_size=16,
    quant_zero=True,
    quant_scale=True,
)
quant_config = QuantConfig(ffn_config=ffn_config, attn_config=attn_config)


# del model

gc.collect
torch.cuda.empty_cache()


## DEFAULT with 4 Experts on GPU

In [7]:
if 'model' in locals():
    del model
model = build_model(
    device=device,
    quant_config=quant_config,
    offload_config=offload_config,
    state_path=state_path,
    routing_strategy="BIASING",
    routing_threshold=0.05
)

/scratch/592241/moe_offload/lib/python3.10/site-packages/torch/nn/init.py:452: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
Loading experts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:04<00:00,  6.67it/s]


In [8]:
from transformers import TextStreamer


tokenizer = AutoTokenizer.from_pretrained(model_name)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
past_key_values = None
sequence = None

seq_len = 0
# while True:
user_input = "Where is Georgia Tech? What is the name of its mascot?"

user_entry = dict(role="user", content=user_input)
input_ids = tokenizer.apply_chat_template([user_entry], return_tensors="pt").to(model.device)

if past_key_values is None:
  attention_mask = torch.ones_like(input_ids)
else:
  seq_len = input_ids.size(1) + past_key_values[0][0][0].size(1)
  attention_mask = torch.ones([1, seq_len - 1], dtype=torch.int, device=model.device)



# sequence = result["sequences"]
# past_key_values = result["past_key_values"]

In [38]:
start_time = time.time()
result = model.generate(
  input_ids=input_ids,
  attention_mask=attention_mask,
  past_key_values=past_key_values,
  streamer=streamer,
  do_sample=True,
  temperature=0.9,
  top_p=0.9,
  min_new_tokens=100,
  max_new_tokens=100,
  pad_token_id=tokenizer.eos_token_id,
  return_dict_in_generate=True,
  # output_hidden_states=True,
  # decoder_router_logits=True, 
  output_router_logits=True,
)
latency = time.time() - start_time
total_experts_saved = 0
for i in result['router_logits'][-32:]:
    total_experts_saved += i[1]
total_experts_saved 

Num_tokens = result['sequences'].shape[1] - input_ids.shape[1]
print(f"Total Latency :{latency} sec, Throughput:{Num_tokens/latency}, Expert Loads saved:{total_experts_saved} ")

Georgia Tech, officially known as the Georgia Institute of Technology, is a public research university located in Atlanta, Georgia, in the southeastern United States. It was established in 1885 and has grown to become a leading institution in technology, engineering, and related sciences.

The mascot of Georgia Tech is called "Buzz." This mascot is a yellow jacket, which is also the nickname and official mascot of Georgia Tech's sports teams.
Total Latency :18.324641704559326 sec, Throughput:5.457132620231214, Expert Loads saved:926 


In [21]:
a = torch.tensor([[-1.6494, -0.2023,  0.3101,  1.6348,  0.3503,  0.4883,  0.0604, -0.3401],[-0.8192, -0.8853, -0.8905, -0.9040, -0.8886, -0.8669, -0.8367, -0.9087]])
b = torch.tensor([-0.8192, -0.8853, -0.8905, -0.9040, -0.8886, -0.8669, -0.8367, -0.9087])

In [23]:
a * b


tensor([[ 1.3512,  0.1791, -0.2761, -1.4779, -0.3113, -0.4233, -0.0505,  0.3090],
        [ 0.6711,  0.7838,  0.7930,  0.8172,  0.7896,  0.7515,  0.7001,  0.8257]])

In [ ]:
def count_not_in_b(A, B):
    return len(set(A.flatten().tolist()) - set(B.flatten().tolist()))

# Example usage
A = torch.tensor([[1, 5]], device='cuda:0') 
B = torch.tensor([[2, 7]], device='cuda:0')
print(count_not_in_b(A, B))  # Output: 3

## Thresholding with 4 Experts on GPU

In [ ]:
start_time = time.time()
result = model.generate(
  input_ids=input_ids.to(device),
  attention_mask=attention_mask.to(device),
  past_key_values=past_key_values,
  streamer=streamer,
  do_sample=True,
  temperature=0.9,
  top_p=0.9,
  min_new_tokens=200,
  max_new_tokens=200,
  pad_token_id=tokenizer.eos_token_id,
  return_dict_in_generate=True,
  output_hidden_states=True,
)
latency = time.time() - start_time

print(f"Total Latency :{latency} sec ")

## Calculate Perplexity

In [ ]:
!pip install -U evaluate
!pip install -U datasets

from evaluate.loading import evaluation_module_factory
from datasets import DownloadConfig, DownloadMode, Version
from utils.perplexity_local_model import Perplexity
from typing import Optional, Union, List


def calculate_perplexity(predictions: List[str], model, tokenizer):
    config_name: Optional[str] = None
    module_type: Optional[str] = None
    process_id: int = 0
    num_process: int = 1
    cache_dir: Optional[str] = None
    experiment_id: Optional[str] = None
    keep_in_memory: bool = False
    download_config: Optional[DownloadConfig] = None
    download_mode: Optional[DownloadMode] = None
    revision: Optional[Union[str, Version]] = None
    
    perplexity_module = evaluation_module_factory(
        "perplexity", module_type=module_type, revision=revision, download_config=download_config, download_mode=download_mode
    )
    
    perplexity = Perplexity(
        config_name=config_name,
        process_id=process_id,
        num_process=num_process,
        cache_dir=cache_dir,
        keep_in_memory=keep_in_memory,
        experiment_id=experiment_id,
        hash=perplexity_module.hash
    )
    
    if module_type and module_type != perplexity.module_type:
        raise TypeError(
            f"No module of module type '{module_type}' not found for 'perplexity' locally, or on the Hugging Face Hub. Found module of module type '{perplexity.module_type}' instead."
        )
    
    # Download and prepare resources for the metric
    perplexity.download_and_prepare(download_config=download_config)
    
    # predictions, model, tokenizer, batch_size: int = 16, add_start_token: bool = True, device=None, max_length=None
    
    return perplexity.compute(predictions=predictions, add_start_token=False, model=model, tokenizer=tokenizer)['mean_perplexity']

calculate_perplexity(["Hello world is a common programming print statement.", "Perplexity is only useful within the same model"], model, tokenizer)

In [ ]:
# With help from GPT
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import DataLoader
from transformers import DataCollatorForLanguageModeling

# Load the C4 dataset


In [ ]:
## Evaluation function
def evaluate_model_on_dataset(model, tokenizer, dataset):
    macro_batch_size = 512
    running_avg_pplx = None
    n = None
    
    for batch in dataset.iter(batch_size=macro_batch_size):
        texts = batch['text']
        next_batch_avg_pplx = calculate_perplexity(texts, model, tokenizer)
        if running_avg_pplx is not None:
            running_avg_pplx = (running_avg_pplx * n + next_batch_avg_pplx * len(texts)) / (n + len(texts))
            n += len(texts)
        else:
            running_avg_pplx = next_batch_avg_pplx
            n = len(texts)
    
    return running_avg_pplx, n


## Get perplexity on C4 using non-instruction fine-tuned model
routing_strategies = ['DEFAULT', 'THRESHOLDING', 'BIASING']

for routing_strategy in routing_strategies:
    dataset = load_dataset("datablations/c4-subsets", split="validation")
    model_name = "mistralai/Mixtral-8x7B-v0.1"
    model = build_model(
        device=device,
        quant_config=quant_config,
        offload_config=offload_config,
        state_path=state_path,
        routing_strategy=routing_strategy,
        routing_threshold=0.05,
        model_name=model_name
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    mean_pplx, n = evaluate_model_on_dataset(model, tokenizer, dataset)
    print(f"{routing_strategy} on C4 dataset | Avg perplexity: {mean_pplx}, n_samples = {n}")

### Using residency information

## Get accuracy on MMLU using instruction fine-tuned model

### Using default routing

### Using thresholding

### Using residency information